In [1]:
import pandas as pd
import glob
from countryboundingboxes import country_bounding_boxes

import ee
import geemap

import geopandas as gpd

import matplotlib.pyplot as plt

## Selecting Bounding Box + Filtering Data

In [2]:
# Compiling CSV sample data to one dataframe
all_csv = glob.glob("./data/*.csv")
df_csv = pd.concat((pd.read_csv(csv) for csv in all_csv))
df_csv

,lat,lon,landcover
0,40.582904,-22.166310,ForestNaturalAreas
1,57.591202,-5.168274,AgriculturalArea
2,70.850927,-9.175052,ForestNaturalAreas
3,37.519676,32.485142,AgriculturalArea
4,50.582377,-7.032158,ArtificialSurfaces
...,...,...,...
4013,45.359292,19.185816,Water
4014,59.992077,16.333949,ForestNaturalAreas
4015,47.464372,10.852922,AgriculturalArea
4016,42.322929,-15.896878,AgriculturalArea


In [3]:
# Selecting country's bounding box
bbox = country_bounding_boxes['NL'][1]

# Extracting min & max longitude and latitude from the bounding box
min_lon = bbox[0]
min_lat = bbox[1]
max_lon = bbox[2]
max_lat = bbox[3]

bbox

(3.31497114423, 50.803721015, 7.09205325687, 53.5104033474)

In [4]:
# Creating mask for filtering CSV data
mask = (
    (df_csv['lat'] >= min_lat) & (df_csv['lat'] <= max_lat) &
    (df_csv['lon'] >= min_lon) & (df_csv['lon'] <= max_lon))


df = df_csv[mask]
df

,lat,lon,landcover
80,53.013246,3.497340,AgriculturalArea
107,51.780871,6.460150,ForestNaturalAreas
179,51.691521,4.668549,ForestNaturalAreas
415,51.876503,4.155372,AgriculturalArea
417,51.104469,5.315605,AgriculturalArea
...,...,...,...
3264,51.938681,3.349975,AgriculturalArea
3570,51.943123,4.140162,AgriculturalArea
3662,52.461430,5.418577,AgriculturalArea
3857,51.163087,4.179379,ForestNaturalAreas


## Converting to Geopandas

In [5]:
geometry = gpd.points_from_xy(df['lon'], df['lat'])
geometry

<GeometryArray>
 ...
Length: 420, dtype: geometry

In [6]:
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs = "EPSG:4326")
gdf

,lat,lon,landcover,geometry
80,53.013246,3.497340,AgriculturalArea,POINT (3.49734 53.01325)
107,51.780871,6.460150,ForestNaturalAreas,POINT (6.46015 51.78087)
179,51.691521,4.668549,ForestNaturalAreas,POINT (4.66855 51.69152)
415,51.876503,4.155372,AgriculturalArea,POINT (4.15537 51.87650)
417,51.104469,5.315605,AgriculturalArea,POINT (5.31560 51.10447)
...,...,...,...,...
3264,51.938681,3.349975,AgriculturalArea,POINT (3.34997 51.93868)
3570,51.943123,4.140162,AgriculturalArea,POINT (4.14016 51.94312)
3662,52.461430,5.418577,AgriculturalArea,POINT (5.41858 52.46143)
3857,51.163087,4.179379,ForestNaturalAreas,POINT (4.17938 51.16309)


## GEE - Authentication

In [7]:
# Trigger the authentication flow.
# ee.Authenticate()

In [8]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


## GEE - Image Collection

In [9]:
# Define the region and time frame
roi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])
startDate = '2023-06-21'
endDate = '2023-09-23'

# Load the Landsat collection
image = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(roi).filterDate(startDate, endDate).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

# Show the number of images
imageCount = image.size().getInfo()
imageCount

100

In [10]:
# Calculate the median image for the entire collection
medianImage = image.reduce(ee.Reducer.median())

In [11]:
# Display the median image
Map = geemap.Map()
Map.centerObject(roi, 12)
Map.addLayer(medianImage, {
    'bands': ['B4_median', 'B3_median', 'B2_median'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}, 'Median Image')


In [12]:
mapbbox = ([min_lon, min_lat], [max_lon, min_lat], [max_lon, max_lat], [min_lon, max_lat])
region = ee.Geometry.Polygon(mapbbox)

In [13]:
# Convert GeoDataFrame to an Earth Engine FeatureCollection
ee_collection = geemap.geopandas_to_ee(gdf)

# Add the FeatureCollection to the map
Map.addLayer(ee_collection, {'color' : 'red'}, 'GeoDataFrame Points')

# Center the map on the GeoDataFrame
Map.centerObject(ee_collection)

# Display the map
# Map

# K-Means

In [16]:
df['landcover'].unique()

array(['AgriculturalArea', 'ForestNaturalAreas', 'ArtificialSurfaces',
       'Water', 'Wetlands'], dtype=object)

In [17]:
training = medianImage.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 500,
    'seed': 0,
    'geometries': True,  # Set this to False to ignore geometries
    'tileScale': 4
})

Map.addLayer(training, {'color':'blue'}, 'training')
# Map

In [18]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the input using the trained clusterer.
result = medianImage.cluster(clusterer)

In [19]:
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(center=[52.16047501732177, 5.20397322430675], controls=(WidgetControl(options=['position', 'transparent_bg…

In [21]:
cluster_samples = result.sampleRegions(**{
    'collection':ee_collection, 
    'properties':['landcover'], 
    'scale':30,
    'tileScale':4
    })

In [24]:
cluster_samples_pd = geemap.ee_to_geopandas(cluster_samples)
cluster_samples_pd

,geometry,cluster,landcover
0,None,3,AgriculturalArea
1,None,0,ForestNaturalAreas
2,None,4,ForestNaturalAreas
3,None,4,AgriculturalArea
4,None,0,AgriculturalArea
...,...,...,...
415,None,2,AgriculturalArea
416,None,1,AgriculturalArea
417,None,4,AgriculturalArea
418,None,0,ForestNaturalAreas


In [ ]:
# Group the data by 'landcover' and select the 'sample' column
landcover_groups = cluster_samples_pd.groupby('landcover')['cluster']

# Create a subplot with one histogram per landcover category
fig, axes = plt.subplots(nrows=len(landcover_groups), figsize=(8, 6))

# Iterate through landcover categories and plot histograms
for i, (landcover, group) in enumerate(landcover_groups):
    ax = axes[i]
    ax.hist(group, bins=20)  # You can adjust the number of bins as needed
    ax.set_title(f'Landcover: {landcover}')
    ax.set_xlabel('Sample Value')
    ax.set_ylabel('Frequency')

# Adjust layout for better visualization
plt.tight_layout()

# Show the histogram plot
plt.show()


In [ ]:
# Convert the clustered image to a GeoDataFrame
clusteredImage = ee.Image(result)

In [ ]:
confusion_matrix = clusteredImage.reduceRegions(
    collection=ee_collection,
    reducer=ee.Reducer.frequencyHistogram(),
    scale=30,
    crs="EPSG:4326"
)

In [ ]:
# Extract 'landcover' values from the FeatureCollection
landcover_values = ee_collection.aggregate_array('landcover')

# Extract cluster labels from the clustered image
cluster_labels = clusteredImage.select(['cluster'])

In [ ]:
# Compute the confusion matrix
confusion_matrix = cluster_labels.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=region,
    scale=30
)

In [ ]:
# Convert the confusion matrix to a dictionary
confusion_dict = confusion_matrix.getInfo()

# Visualize the confusion matrix
print("Confusion Matrix:")
print(confusion_dict)

## Histogram - 5 Clusters

In [ ]:
clusteredImage = ee.Image(result)

In [ ]:
histograms = clusteredImage.reduceRegion(
    reducer=ee.Reducer.histogram(),
    geometry=region,  # Define the region of interest
    scale=30,  # Choose an appropriate scale
    bestEffort=True,
    maxPixels=1e5  # Adjust the maximum number of pixels as needed
)

In [ ]:
histograms.getInfo()

In [ ]:
# Extract the histogram data
hist_data = histograms.get('cluster')
hist_info = hist_data.getInfo()

hist_bucket = hist_info.get('bucketMeans')
hist_values = hist_info.get('histogram')

In [ ]:
hist_bucket

In [ ]:
hist_values

In [ ]:
plt.bar(hist_bucket, hist_values, width=0.5, align='center')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram')
plt.show()